In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Generate Synthetic Data
true_a0 = -0.3  # True intercept
true_a1 = 0.5   # True slope
std_y = 0.2     # Fixed noise standard deviation
num_data_points = 20

np.random.seed(42)
x_data = np.random.uniform(-1, 1, num_data_points)
y_data = true_a0 + true_a1 * x_data + np.random.normal(0, std_y, num_data_points)

# Step 2: Define Grids and Initialize Arrays
num_grid_points = 50
w0_values = np.linspace(-1, 1, num_grid_points)  # Intercept range
w1_values = np.linspace(-1, 1, num_grid_points)  # Slope range

# Initialize 2D arrays for prior, likelihood, and posterior
prior = np.zeros((num_grid_points, num_grid_points))
likelihood = np.zeros((num_grid_points, num_grid_points))
posterior_unnormalized = np.zeros((num_grid_points, num_grid_points))

# Step 3: Initial Prior
std_prior = 0.25
for i in range(num_grid_points):
    for j in range(num_grid_points):
        w0 = w0_values[i]
        w1 = w1_values[j]
        prior[i, j] = (
            (1 / (std_prior * np.sqrt(2 * np.pi))) * np.exp(-(w0**2) / (2 * std_prior**2))
            * (1 / (std_prior * np.sqrt(2 * np.pi))) * np.exp(-(w1**2) / (2 * std_prior**2))
        )

# Step 4: Bayesian Updating Iteratively for Each Data Point
posterior = prior.copy()

for k, (x, y) in enumerate(zip(x_data, y_data)):
    likelihood = np.zeros_like(posterior)
    for i in range(num_grid_points):
        for j in range(num_grid_points):
            w0 = w0_values[i]
            w1 = w1_values[j]
            mu = w0 + w1 * x
            likelihood[i, j] = (
                (1 / (std_y * np.sqrt(2 * np.pi)))
                * np.exp(-((y - mu)**2) / (2 * std_y**2))
            )
    
    # Update posterior
    posterior_unnormalized = likelihood * posterior
    evidence = np.sum(posterior_unnormalized)
    posterior = posterior_unnormalized / evidence  # Normalize

    # Visualization for each iteration
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))

    # Likelihood
    axs[0].contourf(w0_values, w1_values, likelihood.T, cmap='jet',levels=50,)
    axs[0].set_title(f"Likelihood (Iteration {k + 1})")
    axs[0].set_xlabel("w0")
    axs[0].set_ylabel("w1")

    # Posterior
    axs[1].contourf(w0_values, w1_values, posterior.T, cmap='jet',levels=50,)
    axs[1].set_title(f"Posterior (Iteration {k + 1})")
    axs[1].set_xlabel("w0")
    axs[1].set_ylabel("w1")

    # Data Space
    axs[2].scatter(x_data[:k + 1], y_data[:k + 1], edgecolor='blue', facecolor='none', label='Data Points')
    for i in range(5):
        sampled_w0 = np.random.choice(w0_values, p=np.sum(posterior, axis=1))
        sampled_w1 = np.random.choice(w1_values, p=np.sum(posterior, axis=0))
        axs[2].plot([-1, 1], [sampled_w0 + sampled_w1 * (-1), sampled_w0 + sampled_w1 * (1)], color='red', alpha=0.3)
    axs[2].set_title(f"Data Space (Iteration {k + 1})")
    axs[2].set_xlabel("x")
    axs[2].set_ylabel("y")
    axs[2].legend()

    plt.tight_layout()
    plt.show()

# Step 5: Final MAP Estimates
max_idx = np.unravel_index(np.argmax(posterior), posterior.shape)
map_w0 = w0_values[max_idx[0]]
map_w1 = w1_values[max_idx[1]]

print(f"Final MAP estimates: w0 = {map_w0:.3f}, w1 = {map_w1:.3f}")
